# Map, Filter, Reduce - Detroit 911 Calls exercise

## Part 1: Model the Detroit Police Population

In [1]:
import csv

with open("911_Calls_for_Service_(Last_30_Days).csv","rt") as csvfile:
    row_iter = csv.reader(csvfile)
    column_headers = ['X', 'Y', 'incident_id', 'agency', 'incident_address', 'zip_code', 'priority', 'callcode', 'calldescription', 'category', 'call_timestamp', 'precinct_sca', 'respondingunit', 'officerinitiated', 'intaketime', 'dispatchtime', 'traveltime', 'totalresponsetime', 'time_on_scene', 'totaltime', 'neighborhood', 'block_id', 'council_district', 'longitude', 'latitude', 'ObjectId']
    new_list = []
    for i in row_iter:
        d1 = dict()
        for j in range(0,len(column_headers)):
            d1[str(column_headers[j])] = str(i[j])
        new_list.append(d1)

    # Get rid of the column headers
    new_list.pop(0)
    
    # For loop method:
    
    # for i in new_list:
        # if (list(i.keys())[5] == '') or (list(i.keys())[20] == ''):
            # new_list.pop(new_list.index(i))

    # lambda and filter method:
   
    # filters by removing any rows that have zip_codes or neighborhoods with null values
    
    cleaned_list = list(filter((lambda i: (list(i.keys())[5] != '') or (list(i.keys())[20] != '')), new_list))

# Shows the first five rows of the filtered list
print(cleaned_list[0:5])

# Calculate average total response time, average dispatch time, and the average total time using reduce and lambda functions
from functools import reduce

# Avg. response time
total_response_time_values = [int(i['totalresponsetime']) for i in cleaned_list if i['totalresponsetime'].isnumeric()]

sum_trt = reduce(lambda time1, time2: time1 + time2, total_response_time_values)
print(f"\nThe average response time was: {sum_trt/len(total_response_time_values)}")

# Avg. dispatch time
dispatch_time_values = [int(i['dispatchtime']) for i in cleaned_list if i['dispatchtime'].isnumeric()]

sum_dt = reduce(lambda time1, time2: time1 + time2, dispatch_time_values)
print(f"The average dispatch time was: {sum_dt/len(dispatch_time_values)}")

# Avg. total time
total_time_values = [int(i['totaltime']) for i in cleaned_list if i['totaltime'].isnumeric()]

sum_tt = reduce(lambda time1, time2: time1 + time2, total_time_values)
print(f"The average time to complete a call was: {sum_tt/len(total_time_values)}")

[{'X': '-83.0338246217566', 'Y': '42.355185523644', 'incident_id': '202131305618', 'agency': 'DPD ', 'incident_address': 'Scott St & Chene St', 'zip_code': '48207', 'priority': '4', 'callcode': '811040', 'calldescription': 'AUTO X REPORT', 'category': 'ACCREPT ', 'call_timestamp': '2021/11/10 04:28:32+00', 'precinct_sca': '76B ', 'respondingunit': '', 'officerinitiated': 'Yes', 'intaketime': '0', 'dispatchtime': '0', 'traveltime': '0', 'totalresponsetime': '0', 'time_on_scene': '39.6', 'totaltime': '39.6', 'neighborhood': 'Eastern Market', 'block_id': '261635188002036', 'council_district': '5.0', 'longitude': '-83.0338246217566', 'latitude': '42.355185523644', 'ObjectId': '1'}, {'X': '-83.2200838955991', 'Y': '42.4155411581047', 'incident_id': '202131400264', 'agency': 'DPD ', 'incident_address': 'W McNichols Rd & Ashton Ave', 'zip_code': '48219', 'priority': '3', 'callcode': 'SA    ', 'calldescription': 'SPECIAL ATTENTION', 'category': 'SPCL ATT', 'call_timestamp': '2021/11/10 06:26:2

## Part 2: Model the Neighborhood Samples

Using lambda and Map functions, or lambda and Filter, divide the list of dictionaries into smaller lists of dictionaries separated by neighborhood. Using lambda and Reduce, find the average total response time for each neighborhood, the average dispatch time for each neighborhood, and the average total time for each neighborhood and store this into a list of dictionaries. Add a dictionary item to include the population data for all of Detroit in your combined list.

In [2]:
unique_neighborhoods = []
for i in cleaned_list:
    if i["neighborhood"] not in unique_neighborhoods:
        unique_neighborhoods.append(i["neighborhood"])

# List of all unique neighborhoods
unique_neighborhoods

def unique_neighborhood_list(clean_list, uni_neigh_item):
    neigh_list = []
    for i in clean_list:
        if i["neighborhood"] == uni_neigh_item:
            neigh_list.append(i)
    return neigh_list       
    
neighborhoods_dict_list = list(map(lambda i: unique_neighborhood_list(cleaned_list, i), unique_neighborhoods))

The first two neighborhood lists can be seen by running the uncommented version of code below:

In [3]:
# neighborhoods_dict_list[0:2]

In [9]:
# Average time list of dictionaries

def response_times():
    for i in neighborhoods_dict_list:
        return [int(j['totalresponsetime']) for j in i if j['totalresponsetime'].isnumeric()]
    
def dispatch_times():
    for i in neighborhoods_dict_list:
        return [int(j['dispatchtime']) for j in i if j['dispatchtime'].isnumeric()] 


def std(time_list):
    if len(time_list) > 1:
        sum_of_times = reduce(lambda time1, time2: time1 + time2, time_list)
        return sum_of_times/len(time_list)
    # If only one 911 call for a given neighborhood
    elif len(time_list) == 1:
        return time_list[0]

response_times_list = []
for i in neighborhoods_dict_list:
    response_times_list.append([int(j['totalresponsetime']) for j in i if j['totalresponsetime'].isnumeric()])
dispatch_times_list = []
for i in neighborhoods_dict_list:
    dispatch_times_list.append([int(j['dispatchtime']) for j in i if j['dispatchtime'].isnumeric()])

total_times_list = []

for i in neighborhoods_dict_list:
    total_times_list.append([int(j['totaltime']) for j in i if j['totaltime'].isnumeric()])


avg_response_times = list(map(lambda x: std(x), response_times_list))
avg_dispatch_times = list(map(lambda x: std(x), dispatch_times_list))
avg_total_times = list(map(lambda x: std(x), total_times_list))

# Create list of dictionaries for these values

avg_times = []

for i in range(0,len(unique_neighborhoods)):
    avg_times.append({"neighborhood": f"{unique_neighborhoods[i]}","avg_response_time": f"{avg_response_times[i]}",
                      "avg_dispatch_time": f"{avg_dispatch_times[i]}","avg_total_time": f"{avg_total_times[i]}"})

Timmmme to see our results (pun intended):

In [10]:
avg_times[0:10]

[{'neighborhood': 'Eastern Market',
  'avg_response_time': '0.009162303664921465',
  'avg_dispatch_time': '0.02987012987012987',
  'avg_total_time': '17.273291925465838'},
 {'neighborhood': 'Evergreen-Outer Drive',
  'avg_response_time': '2.2088122605363987',
  'avg_dispatch_time': '1.263551401869159',
  'avg_total_time': '30.94'},
 {'neighborhood': 'Midtown',
  'avg_response_time': '0.28268733850129196',
  'avg_dispatch_time': '0.23001547189272822',
  'avg_total_time': '33.647619047619045'},
 {'neighborhood': 'Bethune Community',
  'avg_response_time': '2.4024604569420034',
  'avg_dispatch_time': '1.842756183745583',
  'avg_total_time': '39.87341772151899'},
 {'neighborhood': 'Dexter-Linwood',
  'avg_response_time': '1.6517241379310346',
  'avg_dispatch_time': '0.7003367003367004',
  'avg_total_time': '30.333333333333332'},
 {'neighborhood': 'Rivertown',
  'avg_response_time': '1.1272727272727272',
  'avg_dispatch_time': '1.1818181818181819',
  'avg_total_time': '22.666666666666668'},

## Part 3: Create an Output JSON file

Using the JSON module, format your list of dictionaries as a JSON and test the output with the JSON lint website. Write the tested JSON to a file.

In [25]:
import json
with open("detriot_911_cleaned_list.json","wt+") as file:
    json.dump(cleaned_list, file)
with open("detriot_911_neighborhoods_list.json","wt+") as file:
    json.dump(neighborhoods_dict_list, file)
with open("detriot_911_avg_times.json","wt+") as file:
    json.dump(avg_times, file)